In [3]:
import numpy as np


In [25]:
import numpy as np

class NeuralNetwork:


    def __init__(self, layer_sizes, learning_rate=0.01, seed=42):
        np.random.seed(seed)
        self.lr = learning_rate
        self.W = [np.random.randn(n_prev, n_next) * np.sqrt(2 / n_prev)
                  for n_prev, n_next in zip(layer_sizes[:-1], layer_sizes[1:])]
        self.b = [np.zeros((1, n_next)) for n_next in layer_sizes[1:]]

    #Activation functions

    def _relu(self, x):
        return np.maximum(0, x)

    def _drelu(self, x):
        return (x > 0).astype(float)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _dsigmoid(self, x):
        s = self._sigmoid(x)
        return s * (1 - s)

    #Forward & backward passes

    def _forward(self, X):

        z, a = [], [X]
        for idx, (w, b) in enumerate(zip(self.W, self.b)):
            z_curr = a[-1] @ w + b  # linear step
            if idx == len(self.W) - 1:  # output layer uses sigmoid
                a_curr = self._sigmoid(z_curr)
            else:                       # hidden layers use ReLU
                a_curr = self._relu(z_curr)
            z.append(z_curr)
            a.append(a_curr)
        return z, a

    def _backward(self, z, a, y_true):

        m = y_true.shape[0]
        # Initial gradient (output layer)
        dz = (a[-1] - y_true) * self._dsigmoid(z[-1])
        for i in reversed(range(len(self.W))):
            dw = a[i].T @ dz / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            # Gradient for next layer (if any)
            if i != 0:
                dz = (dz @ self.W[i].T) * self._drelu(z[i-1])
            # Gradient descent update
            self.W[i] -= self.lr * dw
            self.b[i] -= self.lr * db


    def fit(self, X, y, epochs=1000, verbose=True):
        for epoch in range(1, epochs + 1):
            z, a = self._forward(X)
            self._backward(z, a, y)
            if verbose and epoch % (epochs // 10) == 0:
                loss = self._loss(y, a[-1])
                print(f"Epoch {epoch}/{epochs} - loss: {loss:.4f}")

    def predict(self, X):
        _, a = self._forward(X)
        return (a[-1] > 0.5).astype(int)


    def _loss(self, y_true, y_pred):
        """Binary cross-entropy."""
        eps = 1e-12  # prevent log(0)
        y_pred = np.clip(y_pred, eps, 1 - eps)
        m = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred) +
                       (1 - y_true) * np.log(1 - y_pred)) / m


if __name__ == "__main__":
    # XOR demo
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([[0], [1], [1], [0]])

    nn = NeuralNetwork(layer_sizes=[2, 4, 1], learning_rate=0.1)
    nn.fit(X, y, epochs=40000, verbose=False)

    print("Predictions (XOR):")
    for x_i, p_i in zip(X, nn.predict(X)):
        print(f"{x_i} -> {int(p_i)}")


Predictions (XOR):
[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0


/tmp/ipython-input-1614287329.py:91: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"{x_i} -> {int(p_i)}")
